# check torch

In [ ]:
import torch

print("CUDA available?", torch.cuda.is_available())
print("Device count:", torch.cuda.device_count())
print("Device name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU")


In [ ]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


# Prepare Dataset

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import pandas as pd
import os
df = pd.read_pickle('data/trainingData.pkl')
miete_col = 'wohn_leer'

df = df.drop(columns=[miete_col])
X = df.drop(columns=['preis_miet_best','Gastronomy','All_Buildings','Cycleways','Water_Bodies','Fast_Food','Schools','Green_Areas','Building_Types','v_gsm','Buildings_With_Height_Levels','ko_kasskred','v_breitb50','v_lte','Bio-Läden','Landuse','p_markt','p_apo','bev_binw','heiz_wohn_best','v_5g','p_poli','fl_landw','p_freibad','v_breitb1000','Kindergartens','Health','Playgrounds','p_harzt','p_grunds','pfl_ambu','eauto','v_lte','st_einnkr','p_notfall','beschq_insg','Bio_Shops','mitgl_sportv','erw_mini','straft','elterng_v','p_ozmz_oev','kbetr_ue3','bev_entw','p_nelade','bquali_unifh','p_ozmz_miv','p_sek_2','kinder_bg','All_Highways'], axis=1).values

y = df['preis_miet_best'].values

In [ ]:
X.describe()

In [ ]:

# Split
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)

# Torch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).to(device)  # use float32 for regression targets
X_val_tensor = torch.tensor(X_val, dtype=torch.float32).to(device)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).to(device)


#  Create Dataset and DataLoader

In [ ]:
from torch.utils.data import TensorDataset, DataLoader

train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset = TensorDataset(X_val_tensor, y_val_tensor)


# Define the Model Class

In [ ]:
import torch.nn as nn


class TabularNN(nn.Module):
    def __init__(self, input_dim, hidden_dims, dropout_rate, activation_fn):
        super(TabularNN, self).__init__()
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.append(nn.Linear(prev_dim, h_dim))
            if activation_fn == 'relu':
                layers.append(nn.ReLU())
            elif activation_fn == 'tanh':
                layers.append(nn.Tanh())
            layers.append(nn.Dropout(dropout_rate))
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))  # output 1 value for regression
        self.net = nn.Sequential(*layers)

    def forward(self, x):
        return self.net(x).squeeze(1)  # output shape: (batch_size,)



#  Define Training and Evaluation Functions

In [ ]:
def train_one_epoch(model, optimizer, criterion, dataloader, device):
    model.train()
    total_loss = 0
    for X_batch, y_batch in dataloader:
        X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
        #print(f"X_batch device inside train_one_epoch: {X_batch.device}") # Add this line
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        total_loss += loss.item() * X_batch.size(0)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, device):
    model.eval()
    criterion = torch.nn.MSELoss(reduction="sum")  # Sum so we can compute RMSE ourselves
    total_squared_error = 0.0
    total_samples = 0

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.float().to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            total_squared_error += loss.item()
            total_samples += y_batch.size(0)

    mse = total_squared_error / total_samples
    rmse = mse ** 0.5
    return rmse



# Define the Optuna Objective Function

In [ ]:
import optuna

def objective(trial):
    try:
        # sample hyperparams
        hidden_dim1 = trial.suggest_int("hidden_dim1", 32, 256)
        hidden_dim2 = trial.suggest_int("hidden_dim2", 32, 256)
        dropout_rate = trial.suggest_float("dropout_rate", 0.0, 0.5)
        lr = trial.suggest_float("lr", 1e-4, 1e-2, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        activation_fn = trial.suggest_categorical("activation_fn", ["relu", "tanh"])

        model = TabularNN(
            input_dim=52,
            hidden_dims=[hidden_dim1, hidden_dim2],
            dropout_rate=dropout_rate,
            activation_fn=activation_fn,
        ).to(device)

        # DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(10):
            train_one_epoch(model, optimizer, criterion, train_loader, device)
            val_rmse = evaluate(model, val_loader, device)
            trial.report(val_rmse, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise  # Reraise so Optuna logs the failure



# finer 

def objective(trial):
    try:
        hidden_dim1 = trial.suggest_int("hidden_dim1", 100, 130)
        hidden_dim2 = trial.suggest_int("hidden_dim2", 98, 128)
        dropout_rate = trial.suggest_float("dropout_rate", 0.01, 0.05)
        lr = trial.suggest_float("lr", 1e-3, 3e-3, log=True)
        batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
        activation_fn = trial.suggest_categorical("activation_fn", ["relu", "tanh"])

        model = TabularNN(
            input_dim=100,
            hidden_dims=[hidden_dim1, hidden_dim2],
            dropout_rate=dropout_rate,
            activation_fn=activation_fn,
        ).to(device)

        # DataLoader
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        criterion = torch.nn.MSELoss()

        for epoch in range(10):
            train_one_epoch(model, optimizer, criterion, train_loader, device)
            val_rmse = evaluate(model, val_loader, device)
            trial.report(val_rmse, epoch)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return val_rmse

    except Exception as e:
        print(f"Trial failed with exception: {e}")
        raise  # Reraise so Optuna logs the failure



# Launch the Optimization

In [ ]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=500)


In [ ]:
print(X_train_tensor.device)

#  Print and Save Best Result

In [ ]:
print("Best hyperparameters:", study.best_params)
print(f"Best validation accuracy: {1.0 - study.best_value:.4f}")
best_params = study.best_params
best_model = TabularNN(
    input_dim=52,
    hidden_dims=[best_params["hidden_dim1"], best_params["hidden_dim2"]],
    dropout_rate=best_params["dropout_rate"],
    activation_fn=best_params["activation_fn"],
).to(device)

# Dataloader
train_loader = DataLoader(train_dataset, batch_size=best_params["batch_size"], shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=best_params["batch_size"], shuffle=False)

optimizer = torch.optim.Adam(best_model.parameters(), lr=best_params["lr"])
criterion = nn.MSELoss()
# Retrain
for epoch in range(10):
    train_one_epoch(best_model, optimizer, criterion, train_loader, device)

torch.save(best_model.state_dict(), "best_tabular_model_reduced.pt")
